In [235]:

#    Multi-Disciplinary Design Optimisation of a Hydrogen Powered Commercial Aircraft (MDOHPCA)
#    this is the master file that runs each of the individual disciplines 
#    It sets up each of the analyses, then converts it into an openMDAO problem with a certain structure, then solves for minimum mass



In [236]:
import openmdao.api as om
import pandas as pd
import numpy as np

In [237]:
class MDOHPCA(om.Group):
    """
    Top level group containing the MDO.
    """           
    def setup(self):
        """
        set up each person's disciplines that can contain analyses
        """
        self.add_subsystem('aero', aero())
        self.add_subsystem('sys', sys())
        self.add_subsystem('stab', stab())
        self.add_subsystem('struct', struct())
        
        self.add_subsystem('mass', mass())
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually)
        self.promotes('sys',any=['*'])
        self.promotes('struct',any=['*'])
        
        self.promotes('mass',any=['*'])

In [238]:
class mass(om.ExplicitComponent):
    def setup(self):
        self.add_input('systems_mass')
        self.add_input('weight_structures')
        
        self.add_output('m')
        prob.model.add_objective('m')
        
        #prob.model.add_constraint('m', lower = 20000)

    def compute(self,inputs,outputs):
        outputs['m'] = inputs['systems_mass'] + inputs['weight_structures']

In [239]:
%run struct/structsetup.ipynb
#%run stab/stabsetup.ipynb
%run systems/systemssetup.ipynb
class aero(om.ExplicitComponent):
    pass
class stab(om.ExplicitComponent):
    pass

In [240]:
prob = om.Problem(model = MDOHPCA())
#prob.model.add_subsystem('aero', om.Group())

In [241]:
#setting inputs to reduce ambiguities where inputs differ in different disciplines
prob.model.set_input_defaults('CL', val = 0.6)
prob.model.set_input_defaults('sweep', val = 15)
prob.model.set_input_defaults('c', val = 4)
prob.model.set_input_defaults('b', val = 60)
prob.model.set_input_defaults('root_x', val = 10)
prob.model.set_input_defaults('m', val = 240000)


prob.model.add_design_var('fuselage_diameter')

In [242]:
#declare problem options (driver, optimiser)
prob.model.driver = om.ScipyOptimizeDriver()
prob.model.driver.options['optimizer'] = 'SLSQP'
#prob.nonlinear_solver = om.NonlinearBlockGS()
#prob.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.nonlinear_solver.options['iprint'] = 2
prob.model.nonlinear_solver.options['maxiter'] = 50

#prob.nonlinear_solver.options['maxiter'] = 100
prob.model.driver.options['maxiter'] = 5
prob.model.driver.options['tol'] = 1e-8

#setup problem to be run
prob.setup()

In [243]:
#%run aero/aerosetup.ipynb

In [244]:
prob.run_model()

NL: NLBGS 1 ; 3.91418804e+11 1
NL: NLBGS 2 ; 5.08448154e+09 0.0129898755
NL: NLBGS 3 ; 7.96919994e+09 0.020359778
NL: NLBGS 4 ; 6.90111385e+09 0.0176310228
NL: NLBGS 5 ; 4.48047641e+09 0.0114467582
NL: NLBGS 6 ; 2.52474567e+09 0.00645024114
NL: NLBGS 7 ; 1.32714935e+09 0.00339061215
NL: NLBGS 8 ; 674218596 0.00172249925
NL: NLBGS 9 ; 336823294 0.000860518939
NL: NLBGS 10 ; 166888657 0.000426368521
NL: NLBGS 11 ; 82355724.5 0.000210403087
NL: NLBGS 12 ; 40559897.9 0.000103622763
NL: NLBGS 13 ; 19956082 5.09839636e-05
NL: NLBGS 14 ; 9813975.88 2.50728268e-05
NL: NLBGS 15 ; 4825164.19 1.23273694e-05
NL: NLBGS 16 ; 2372076.95 6.06020184e-06
NL: NLBGS 17 ; 1166059.37 2.97905813e-06
NL: NLBGS 18 ; 573192.327 1.4643965e-06
NL: NLBGS 19 ; 281756.59 7.19834068e-07
NL: NLBGS 20 ; 138498.431 3.53836937e-07
NL: NLBGS 21 ; 68079.158 1.73929196e-07
NL: NLBGS 22 ; 33464.3804 8.54950761e-08
NL: NLBGS 23 ; 16449.4374 4.20251589e-08
NL: NLBGS 24 ; 8085.72821 2.06574854e-08
NL: NLBGS 25 ; 3974.54254 1.01

In [245]:
prob.model.list_outputs()

28 Explicit Output(s) in 'model'

varname                         val                 
------------------------------  --------------------
sys
  fuel_mass_calc
    fuel_mass                   [10003.1466033]     
  tanks
    tank1_mass_full             [8423.92841006]     
    tank1_mass_empty            [2422.04044808]     
    tank2_mass_full             [6077.63352833]     
    tank2_mass_empty            [2076.37488701]     
    CGtp                        [30.34823787]       
    tank1_x                     [5.79866734]        
    tank2_x                     [49.06388984]       
    length_fuselage             [60.530445]         
  pipes
    pipe_CG                     [23.64889682]       
    pipe_mass                   [76.49718754]       
  engine
    engine_mass                 [67.26137349]       
    engine_x                    [14.82308546]       
    engine_y                    [18.]               
  actuator
    each_flap_actuator_mass     [2.55]              
    each

[('sys.fuel_mass_calc.fuel_mass', {'val': array([10003.1466033])}),
 ('sys.tanks.tank1_mass_full', {'val': array([8423.92841006])}),
 ('sys.tanks.tank1_mass_empty', {'val': array([2422.04044808])}),
 ('sys.tanks.tank2_mass_full', {'val': array([6077.63352833])}),
 ('sys.tanks.tank2_mass_empty', {'val': array([2076.37488701])}),
 ('sys.tanks.CGtp', {'val': array([30.34823787])}),
 ('sys.tanks.tank1_x', {'val': array([5.79866734])}),
 ('sys.tanks.tank2_x', {'val': array([49.06388984])}),
 ('sys.tanks.length_fuselage', {'val': array([60.530445])}),
 ('sys.pipes.pipe_CG', {'val': array([23.64889682])}),
 ('sys.pipes.pipe_mass', {'val': array([76.49718754])}),
 ('sys.engine.engine_mass', {'val': array([67.26137349])}),
 ('sys.engine.engine_x', {'val': array([14.82308546])}),
 ('sys.engine.engine_y', {'val': array([18.])}),
 ('sys.actuator.each_flap_actuator_mass', {'val': array([2.55])}),
 ('sys.actuator.each_aileron_actuator_mass', {'val': array([0.1275])}),
 ('sys.actuator.actuators_CG_x'

In [246]:
prob.model.get_constraints()

{'sys.fuel_mass_calc.fuel_mass': {'name': 'fuel_mass',
  'alias': None,
  'lower': 20000.0,
  'upper': 1e+30,
  'equals': None,
  'linear': False,
  'indices': None,
  'scaler': None,
  'adder': None,
  'ref': None,
  'ref0': None,
  'type': 'con',
  'units': None,
  'cache_linear_solution': False,
  'parallel_deriv_color': None,
  'flat_indices': False,
  'source': 'sys.fuel_mass_calc.fuel_mass',
  'distributed': False,
  'total_scaler': None,
  'total_adder': None,
  'size': 1,
  'global_size': 1},
 'struct.landing_gear_calcs.landing_tension': {'name': 'landing_tension',
  'alias': None,
  'lower': -1e+30,
  'upper': 9000000.0,
  'equals': None,
  'linear': False,
  'indices': None,
  'scaler': None,
  'adder': None,
  'ref': None,
  'ref0': None,
  'type': 'con',
  'units': None,
  'cache_linear_solution': False,
  'parallel_deriv_color': None,
  'flat_indices': False,
  'source': 'struct.landing_gear_calcs.landing_tension',
  'distributed': False,
  'total_scaler': None,
  'total_a

In [247]:
#get properties of specific constants in disciplines
prob.model.sys.options._dict['pp_mass']

{'val': 32816.0,
 'values': None,
 'types': None,
 'desc': 'people and cargo mass',
 'upper': None,
 'lower': None,
 'check_valid': None,
 'has_been_set': True,
 'allow_none': False,
 'recordable': True,
 'deprecation': None}